In [6]:
%pip install langchain langchain-text-splitters langchain-openai langchain-community langchain-core chromadb pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.storage import InMemoryByteStore
from langchain.retrievers import ParentDocumentRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import os


In [ ]:
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"  # Substitua pela sua chave da OpenAI


In [9]:
embeddings_model = OpenAIEmbeddings()

llm = ChatOpenAI(model_name="gpt-4o", max_tokens=400)

In [10]:
pdf_path = "os-sertoes.pdf"

load_pdf = PyPDFLoader(pdf_path, extract_images=False)

pages = load_pdf.load_and_split()

In [11]:
# Child splitter - chunks pequenos para busca precisa
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200
)

# Parent splitter - chunks grandes para contexto completo
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=4000,
    chunk_overlap=200,
    length_function=len,
    add_start_index=True
)

In [12]:
store = InMemoryByteStore()

vectorstore = Chroma(embedding_function=embeddings_model)

C:\Users\Renan\AppData\Local\Temp\ipykernel_16340\3484065597.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(embedding_function=embeddings_model)


In [13]:
# Criar o Parent Document Retriever
parent_document_retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

# Adicionar documentos em lotes para evitar erro de batch size
batch_size = 10  # Processar 10 páginas por vez
total_pages = len(pages)

for i in range(0, total_pages, batch_size):
    batch = pages[i:i + batch_size]
    parent_document_retriever.add_documents(batch, ids=None)
    print(f"  ✓ Processadas {min(i + batch_size, total_pages)}/{total_pages} páginas")

print(f"\n✓ Total de {total_pages} páginas processadas com sucesso!")


  ✓ Processadas 10/658 páginas
  ✓ Processadas 20/658 páginas
  ✓ Processadas 20/658 páginas
  ✓ Processadas 30/658 páginas
  ✓ Processadas 30/658 páginas
  ✓ Processadas 40/658 páginas
  ✓ Processadas 40/658 páginas
  ✓ Processadas 50/658 páginas
  ✓ Processadas 50/658 páginas
  ✓ Processadas 60/658 páginas
  ✓ Processadas 60/658 páginas
  ✓ Processadas 70/658 páginas
  ✓ Processadas 70/658 páginas
  ✓ Processadas 80/658 páginas
  ✓ Processadas 80/658 páginas
  ✓ Processadas 90/658 páginas
  ✓ Processadas 90/658 páginas
  ✓ Processadas 100/658 páginas
  ✓ Processadas 100/658 páginas
  ✓ Processadas 110/658 páginas
  ✓ Processadas 110/658 páginas
  ✓ Processadas 120/658 páginas
  ✓ Processadas 120/658 páginas
  ✓ Processadas 130/658 páginas
  ✓ Processadas 130/658 páginas
  ✓ Processadas 140/658 páginas
  ✓ Processadas 140/658 páginas
  ✓ Processadas 150/658 páginas
  ✓ Processadas 150/658 páginas
  ✓ Processadas 160/658 páginas
  ✓ Processadas 160/658 páginas
  ✓ Processadas 170/658 p

In [14]:
# Criar o template do prompt
TEMPLATE = """Você é um assistente especializado em literatura brasileira.
Use o seguinte contexto para responder à pergunta de forma detalhada e precisa.

Contexto:
{context}

Pergunta: {question}

Resposta detalhada:"""

prompt = ChatPromptTemplate.from_template(TEMPLATE)

# Função auxiliar para formatar documentos
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# Criar a chain RAG com Parent Document Retriever
parent_rag_chain = (
    {
        "context": parent_document_retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)



In [21]:
# Pergunta 1
pergunta = "Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?"



resposta = parent_rag_chain.invoke(pergunta)

print(resposta)


Euclides da Cunha, em sua obra "Os Sertões", oferece uma visão detalhada e muitas vezes sombria do sertão nordestino, descrevendo-o como uma região marcada por desafios ambientais extremos que impactam profundamente a vida de seus habitantes. A caracterização do ambiente natural no sertão é complexa e multifacetada, concentrando-se em três aspectos principais:

1. **Aridez e Hostilidade do Terreno**: Euclides retrata o sertão nordestino como uma terra inóspita, onde a aridez predomina. A região é descrita como um "deserto" marcado por secos e longos períodos de estiagem que trazem sofrimentos não apenas para os seres humanos, mas para toda a economia local. Ele destaca que o martírio do homem no sertão é refletido pelo sofrimento de uma terra que parece condenada a uma eternidade de secura e escassez. Essa realidade impõe um desafio constante aos habitantes, que precisam adaptar sua vida às duras condições climáticas.

2. **Impacto na Vida Humana**: As condições naturais adversas do se

In [22]:
# Pergunta 2
pergunta = "Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?"


resposta = parent_rag_chain.invoke(pergunta)

print(resposta)



Em "Os Sertões", Euclides da Cunha faz uma descrição minuciosa da população sertaneja, enfatizando sua resiliência, adaptabilidade e vigor. O autor analisa essa população dentro do contexto adverso do sertão brasileiro, destacando a influência do meio ambiente na formação das características dos habitantes.

### Características da População Sertaneja:

1. **Resiliência e Robustez**: Euclides observa que a população sertaneja é incrivelmente resistente e robusta. Essa resiliência é um resultado direto das condições áridas e difíceis do sertão. Os sertanejos são descritos como sobreviventes natos, acostumados a lutar contra a escassez de água e alimentos, e a enfrentar a inclemência do clima.

2. **Adaptabilidade**: A capacidade de adaptação dos sertanejos é outro ponto central. Eles desenvolveram modos de vida que se alinham com as condições do sertão, ajustando suas práticas agrícolas e de subsistência às variações do clima e do solo.

3. **Simplicidade e Dureza**: O caráter simples e 

In [20]:
# Pergunta 3
pergunta = "Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?"


resposta = parent_rag_chain.invoke(pergunta)

print(resposta)


O contexto histórico e político que levou à Guerra de Canudos, conforme descrito por Euclides da Cunha, foi marcado por uma série de elementos socioeconômicos, culturais e políticos complexos no Brasil do final do século XIX. A Guerra de Canudos ocorreu entre 1896 e 1897, em um momento de transição e instabilidade para o recém-formado regime republicano brasileiro, que substituiu o Império em 1889.

1. **Transição Política e Insegurança do Regime Republicano**: Após a Proclamação da República em 1889, o Brasil ainda buscava consolidar suas instituições republicanas. O novo governo enfrentava desafios para estabelecer e manter sua autoridade, especialmente em regiões distantes dos grandes centros urbanos, como o sertão nordestino, onde tradicionalismos e lealdades ao antigo regime monárquico ainda eram fortes. Havia uma percepção do governo de que movimentos populares, como a comunidade de Canudos, poderiam representar ameaças monarquistas ao regime republicano.

2. **Figura de Antônio 

In [23]:
# Pergunta 4
pergunta = "Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?"

resposta = parent_rag_chain.invoke(pergunta)

print(resposta)


Euclides da Cunha, em sua obra "Os Sertões", oferece uma descrição abrangente e multifacetada de Antônio Vicente Mendes Maciel, mais conhecido como Antônio Conselheiro. Conselheiro é retratado como uma figura carismática e religiosa que exercia grande influência sobre os sertanejos do Nordeste brasileiro. Nascido em Quixeramobim, no Ceará, ele percorreu longas distâncias no sertão da Bahia, dedicando-se à construção de igrejas e à reforma de cemitérios, ações que o tornaram conhecido e respeitado na região.

No contexto da Guerra de Canudos, Antônio Conselheiro é apresentado como o líder espiritual e comunitário da comunidade de Belo Monte, às margens do Rio Vaza-Barris, em Canudos. Euclides da Cunha destaca a imagem do Conselheiro como um profeta e um visionário, que conseguiu reunir em torno de si uma comunidade de sertanejos pobres, atraídos por sua mensagem de fé, esperança e justiça social em um cenário de grande miséria e adversidade.

Euclides descreve a figura de Antônio Consel

In [24]:
# Pergunta 5
pergunta = "Quais são os principais aspectos da crítica social e política presentes em Os Sertões? Como esses aspectos refletem a visão do autor sobre o Brasil da época?"


resposta = parent_rag_chain.invoke(pergunta)

print(resposta)


"Os Sertões" de Euclides da Cunha é uma obra seminal que combina análise científica, histórica e literária para abordar a Campanha de Canudos, um conflito que simboliza a tensão entre o Brasil "civilizado" do litoral e o Brasil "primitivo" do sertão. Os principais aspectos da crítica social e política presentes na obra incluem:

1. **Confronto Civilização vs. Barbárie**: Euclides da Cunha apresenta o sertão e seus habitantes, os sertanejos, sob a lente de quem vê uma dicotomia entre a civilização litorânea e a suposta barbárie do interior. Ele critica a visão simplista e preconceituosa da elite letrada da época que desconsidera as complexidades culturais e sociais do sertão. Essa crítica reflete uma percepção da época, em que o desenvolvimento era frequentemente medido pelas normas culturais e sociais do litoral urbanizado.

2. **Desigualdade Social e Econômica**: O livro pinta um quadro detalhado das condições abjetas de vida dos sertanejos, que enfrentam pobreza extrema, secas freque